In [ ]:
import sqlite3
import pandas as pd
import datetime

conn = sqlite3.connect('../database.sqlite')
query_matches = \
    'select match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, date \
    from Match order by id limit 3000;'
matches = pd.read_sql(query_matches, conn, index_col='match_api_id', parse_dates=['date']);
matches.head(6)

In [ ]:
def teamattrs(team_api_id, matchdate):
    """
    Helper method that gets the specified team's attributes from
    the date closest to matchdate.
    
    Keyword arguments:
    team_api_id -- the api_id of the team of interest (Integer)
    matchdate -- pd.Timestamp
    """
    query = \
        'select date, buildUpPlaySpeed, buildUpPlayPassing, \
                chanceCreationPassing, chanceCreationCrossing, chanceCreationShooting, \
                defencePressure, defenceAggression, defenceTeamWidth \
        from Team_Attributes \
        where team_api_id = %s \
        order by date asc;' % team_api_id
    attrs = pd.read_sql(query, conn, parse_dates=['date'])
    if (not attrs.empty):
        closestattrs = None
        mindiff = None
        for index, row in attrs.iterrows():
            date = row['date']
            diff = abs(matchdate - date)
            if (mindiff is None or diff < mindiff):
                mindiff = diff
                closestattrs = row
        return closestattrs.drop('date')
    else:
        return None
    
print('Example output from teamattrs:\n')
teamattrs(9993, pd.to_datetime('2008-08-16'))

In [ ]:
def differencevector(row):
    """
    This function is applied to each row in the 'matches' DataFrame.
    For each row, it returns a pandas.Series containing the number of
    goals scored by each team and the result from the home team's
    perspective.
    
    Keyword arguments:
    row -- pandas.Series
    """
    date = row['date']
    homegoal = row['home_team_goal']
    homeattrs = teamattrs(row['home_team_api_id'], date)
    
    awaygoal = row['away_team_goal']
    awayattrs = teamattrs(row['away_team_api_id'], date)
    
    if (homeattrs is not None and awayattrs is not None):
        differencevector = homeattrs - awayattrs

        if (homegoal > awaygoal):
            result = 2
        elif (awaygoal >= homegoal):
            result = 0
        else:
            result = 1

        results = pd.Series({
            'home_team_api_id': row['home_team_api_id'],
            'away_team_api_id': row['away_team_api_id'],
            'outcome':  result
        })
        return results.append([differencevector])
    else:
        return

results = matches.apply(differencevector, axis=1).dropna(axis=0, how='all')
conn.close()
results

In [ ]:
from sklearn.tree import DecisionTreeClassifier

features = [
    'buildUpPlaySpeed',
    'buildUpPlayPassing',
    'chanceCreationPassing',
    'chanceCreationCrossing',
    'chanceCreationShooting',
    'defencePressure',
    'defenceAggression',
    'defenceTeamWidth'
]

x = results[features]
y = results['outcome']

dt = DecisionTreeClassifier()
dt = dt.fit(x, y)